In [1]:
import os
import cv2
import dlib
import numpy as np
import pickle

# Chemin du répertoire contenant les dossiers d'images
base_directory = r".\data"

# Initialisation des modèles dlib
pose_predictor_68_point = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
face_detector = dlib.get_frontal_face_detector()

# Initialiser les listes pour stocker les embeddings et les étiquettes
embeddings = []
labels = []

def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image non lue : {image_path}")
    return image

def extract_face_embeddings(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    if len(faces) == 0:
        return None
    face = faces[0]
    shape = pose_predictor_68_point(image, face)
    face_descriptor = face_encoder.compute_face_descriptor(image, shape)
    return np.array(face_descriptor)

# Créer un dictionnaire pour attribuer un ID unique à chaque personne sans trier
person_names = os.listdir(base_directory)  # Obtenir les noms de dossiers sans les trier
person_ids = {name: idx + 1 for idx, name in enumerate(person_names)}

# Lire les images à partir des répertoires
for person_name, person_id in person_ids.items():
    person_folder = os.path.join(base_directory, person_name)
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            image = load_image(image_path)
            if image is not None:
                face_embedding = extract_face_embeddings(image)
                if face_embedding is not None:
                    embeddings.append(face_embedding)
                    labels.append(person_id)  # Utiliser l'ID de la personne comme étiquette
                else:
                    print(f"Aucun visage détecté dans l'image : {image_path}")

# Convertir les données en numpy arrays
embeddings = np.array(embeddings)
labels = np.array(labels)

# Enregistrer les embeddings et les étiquettes
with open('embeddings.pkl', 'wb') as f:
    pickle.dump((embeddings, labels), f)

print("Embeddings et étiquettes sauvegardés avec succès.")


Aucun visage détecté dans l'image : .\data\Adriana Lima\Adriana Lima30_153.jpg
Aucun visage détecté dans l'image : .\data\Alex Lawther\Alex Lawther112_12.jpg
Aucun visage détecté dans l'image : .\data\Alex Lawther\Alex Lawther135_28.jpg
Aucun visage détecté dans l'image : .\data\Alex Lawther\Alex Lawther188_66.jpg
Aucun visage détecté dans l'image : .\data\Alex Lawther\Alex Lawther194_68.jpg
Aucun visage détecté dans l'image : .\data\Alex Lawther\Alex Lawther91_144.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte142_179.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte159_197.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte209_219.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte215_222.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte31_241.jpg
Aucun visage détecté dans l'image : .\data\Alvaro Morte\Alvaro Morte7_267.jpg
Aucun visage détecté dans l'image : .\data\alycia

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pickle

# Charger les embeddings et les étiquettes
with open('embeddings.pkl', 'rb') as f:
    embeddings, labels = pickle.load(f)

# Encoder les étiquettes
le = LabelEncoder()
labels = le.fit_transform(labels)

# Diviser les données en ensembles d'entraînement et de test
(train_data, test_data, train_labels, test_labels) = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Entraîner un classificateur SVM
clf = SVC(kernel='linear', probability=True)
clf.fit(train_data, train_labels)

# Enregistrer le classificateur et le LabelEncoder
with open('classifier.pkl', 'wb') as f:
    pickle.dump((clf, le), f)

In [5]:
from sklearn.metrics import classification_report, accuracy_score
import pickle

# Charger le classificateur et le LabelEncoder
with open('classifier.pkl', 'rb') as f:
    clf, le = pickle.load(f)

# Charger les embeddings et les étiquettes
with open('embeddings.pkl', 'rb') as f:
    embeddings, labels = pickle.load(f)

# Encoder les étiquettes
labels = le.transform(labels)

# Diviser les données en ensembles d'entraînement et de test
(train_data, test_data, train_labels, test_labels) = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Effectuer des prédictions sur l'ensemble de test
predictions = clf.predict(test_data)

# Calculer l'accuracy totale
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy totale: {accuracy:.2f}")

# Générer un rapport de classification pour afficher la précision de chaque personne (chaque ID)
report = classification_report(test_labels, predictions, target_names=[str(i) for i in le.classes_])
print(report)


Accuracy totale: 0.99
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        47
           2       0.97      1.00      0.99        36
           3       1.00      1.00      1.00        46
           4       0.95      1.00      0.98        20
           5       0.97      1.00      0.99        38
           6       1.00      1.00      1.00        27
           7       1.00      0.97      0.99        40
           8       1.00      1.00      1.00        39
           9       1.00      0.98      0.99        46
          10       1.00      0.96      0.98        23
          11       0.95      1.00      0.97        19
          12       1.00      1.00      1.00        20
          13       1.00      1.00      1.00        41
          14       1.00      1.00      1.00        25
          15       0.95      1.00      0.98        20
          16       1.00      1.00      1.00        29
          17       0.95      1.00      0.97        39
     

In [13]:
import cv2
import dlib
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# Charger le classificateur et le LabelEncoder
with open('classifier.pkl', 'rb') as f:
    clf, le = pickle.load(f)

# Initialiser les modèles dlib
pose_predictor_68_point = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
face_detector = dlib.get_frontal_face_detector()

# Fonction pour extraire les embeddings de visage avec dlib
def extract_face_embeddings(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray, 1)
    if len(faces) == 0:
        return None
    else:
        face = faces[0]
        shape = pose_predictor_68_point(image, face)
        face_descriptor = face_encoder.compute_face_descriptor(image, shape)
        return np.array(face_descriptor)

# Capture d'images depuis la caméra
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Réduire la taille de la frame pour un traitement plus rapide
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    face_embedding = extract_face_embeddings(small_frame)
    if face_embedding is not None:
        face_embedding = face_embedding.reshape(1, -1)
        predictions = clf.predict_proba(face_embedding)[0]
        max_index = np.argmax(predictions)
        if predictions[max_index] > 0.5:
            person_id = le.inverse_transform([max_index])[0]
            label_text = f"ID: {person_id}"
        else:
            label_text = "Inconnu"
        # Afficher le résultat sur la frame originale
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Facial Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
# Afficher chaque nom avec son ID
for person_name, person_id in person_ids.items():
    print(f"Nom: {person_name}, ID: {person_id}")


Nom: Adriana Lima, ID: 1
Nom: Alex Lawther, ID: 2
Nom: Alexandra Daddario, ID: 3
Nom: Alvaro Morte, ID: 4
Nom: alycia dabnem carey, ID: 5
Nom: Amanda Crew, ID: 6
Nom: amber heard, ID: 7
Nom: Andy Samberg, ID: 8
Nom: Anne Hathaway, ID: 9
Nom: Anthony Mackie, ID: 10
Nom: Avril Lavigne, ID: 11
Nom: barack obama, ID: 12
Nom: barbara palvin, ID: 13
Nom: Ben Affleck, ID: 14
Nom: Bill Gates, ID: 15
Nom: Bobby Morley, ID: 16
Nom: Brenton Thwaites, ID: 17
Nom: Brian J. Smith, ID: 18
Nom: Brie Larson, ID: 19
Nom: camila mendes, ID: 20
Nom: Chris Evans, ID: 21
Nom: Chris Hemsworth, ID: 22
Nom: Chris Pratt, ID: 23
Nom: Christian Bale, ID: 24
Nom: Cristiano Ronaldo, ID: 25
Nom: Danielle Panabaker, ID: 26
Nom: david letterman, ID: 27
Nom: Dominic Purcell, ID: 28
Nom: Dwayne Johnson, ID: 29
Nom: Eliza Taylor, ID: 30
Nom: Elizabeth Lail, ID: 31
Nom: elizabeth olsen, ID: 32
Nom: ellen page, ID: 33
Nom: elon musk, ID: 34
Nom: Emilia Clarke, ID: 35
Nom: Emma Stone, ID: 36
Nom: Emma Watson, ID: 37
Nom: ga